<a href="https://colab.research.google.com/github/Anaa1101/MediRag/blob/main/MediRag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
b!pip install langchain openai faiss-cpu sentence-transformers PyMuPDF


# New Section

getpass


1. INSTALLATION


In [ ]:
pip install langchain langchain-community langchain-text-splitters

In [ ]:
!pip install langchain-community pypdf

In [ ]:
pip install chromadb

In [ ]:
pip install -U langchain-google-genai


In [ ]:
pip install langchain-chroma

In [ ]:
pip install pypdf

In [ ]:
 pip install langchain-google-genai

2.UPLOADING AND LOADING MY PDF


In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print("Uploaded file:", file_name)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/" + file_name)
docs=loader.load()
print("no of pages",len(docs));
print("first page sample",docs[0].page_content[:500])

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50

)
chunks=splitter.split_documents(docs)

print(f"Total chunks:{len(chunks)}")
print("Sample chunk: ",chunks[1].page_content[:300])

making embeddings:


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="model/embedding-004")

Store in Chroma


In [ ]:
import os
from getpass import getpass
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",   # ✅ correct model
    google_api_key=os.environ["GOOGLE_API_KEY"]
)


In [ ]:
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
retriever=vectorstore.as_retriever()

In [ ]:
from langchain_google_genai  import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

template="""u are a helpful medical assistant.
If the user's question is vague or incomplete, try to interpret it
based on the context. If you still cannot, politely ask them to clarify.

Context: {context}
Question: {question}

Answer and then suggest a clear follow-up question:"""

prompt=PromptTemplate(
    template=template,
    input_variables=["context","question"] #placeholders
)



qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)
print("Ask me anything about medical data, research, or equipment!")
while True:
  query=input()
  if query.lower() in["bye","exit","quit","q"]:
    print("Goodbye 👋")
    break
  response=qa_chain.run(query)
  print("\nAnswer:",response)